## **Big Data Engineering Group Assignment** 
### **Salmoli Chandra 24PGAI0057**
### **Meera Karamta 24PGAI0003**
### **Kishnu Srivastava 24PGAI0042**
### **Praveen Kumar Singh 24PGAI0068**

In [0]:
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.2 
 Master 
 local[8] 
 AppName 
 Databricks Shell


### Install Confluent Libraries

```
confluent-kafka[avro,json,protobuf]>=1.4.2

```

## Streaming data from confluent servers

#### Endpoints
```
Bootstrap server	: pkc-12576z.us-west2.gcp.confluent.cloud:9092
```

ConfluentIO Adaptor

```
Select output record value format: JSON
Select a template: RATINGS
```


# Stream Ingestion

In [0]:
# Subscribe to 1 topic

confluentApiKey = 'VO4DBIVRWZQV6V4Q'

confluentSecret = 'Y5wqrwJTNldowy6/R6RqAgStwYc/I4GREAhmW+1BEkVyaphYevD0x01ttllpFcv2'

confluentTopicName = 'topic_0'

stream_df = ( spark
             .readStream
             .format("kafka")
             .option("kafka.bootstrap.servers", "pkc-12576z.us-west2.gcp.confluent.cloud:9092")
             .option("kafka.security.protocol", "SASL_SSL")
             .option("kafka.sasl.jaas.config", "kafkashaded.org.apache.kafka.common.security.plain.PlainLoginModule required username='{}' password='{}';".format(confluentApiKey, confluentSecret))
             .option("kafka.ssl.endpoint.identification.algorithm", "https")
             .option("kafka.sasl.mechanism", "PLAIN")
             .option("subscribe", confluentTopicName)
             .option("startingOffsets", "earliest")
             .option("maxOffsetsPerTrigger", 10)
             .option("failOnDataLoss", "false")
             .load() )

message_df = stream_df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")

In [0]:
from pyspark.sql.types import StructType, StructField, DoubleType, IntegerType, StringType
from pyspark.sql.functions import col, from_json, split
from pyspark.sql.types import StructType, StructField, IntegerType, StringType


schema = StructType([
    StructField("rating_id", IntegerType(), True),
    StructField("user_id", IntegerType(), True),
    StructField("stars", IntegerType(), True),
    StructField("route_id", IntegerType(), True),
    StructField("rating_time", IntegerType(), True),
    StructField("channel", StringType(), True),
    StructField("message", StringType(), True)
])
    

new_rating_df = message_df.select (from_json("value",schema).alias("data")).select(col("data.*"))

In [0]:
new_rating_df = new_rating_df.withColumn("rating_time", expr("from_unixtime(unix_timestamp('2000-01-01 00:00:00') + rating_time*3600)"))

In [0]:
display(new_rating_df)

rating_id,user_id,stars,route_id,rating_time,channel,message
6,4,2,3120,2000-01-03 13:00:00,iOS-test,why is it so difficult to keep the bathrooms clean ?
3,19,2,2118,2000-01-02 01:00:00,android,more peanuts please
2,17,4,5719,2000-01-01 13:00:00,iOS-test,Exceeded all my expectations. Thank you !
1,3,4,5548,2000-01-01 01:00:00,ios,"airport refurb looks great, will fly outta here more!"
4,-1,2,4632,2000-01-02 13:00:00,web,"Surprisingly good, maybe you are getting your mojo back at long last!"
5,15,2,7752,2000-01-03 01:00:00,iOS-test,worst. flight. ever. #neveragain
10,13,1,6696,2000-01-05 13:00:00,iOS-test,"airport refurb looks great, will fly outta here more!"
9,5,4,2732,2000-01-05 01:00:00,iOS,why is it so difficult to keep the bathrooms clean ?
36,19,4,7747,2000-01-18 13:00:00,iOS,"thank you for the most friendly, helpful experience today at your new lounge"
7,15,2,7178,2000-01-04 01:00:00,iOS-test,your team here rocks!


In [0]:
from pyspark.sql.functions import *

spark.conf.set("spark.sql.shuffle.partitions", "2")

usercount_df = new_rating_df.groupby(col("stars")).count()

query = (
  usercount_df
    .writeStream
    .trigger(processingTime='5 seconds')
    .format("memory")        # memory = store in-memory table (for testing only in Spark 2.0)
    .queryName("nstarcounts")     # counts = name of the in-memory table
    .outputMode("complete")  # complete = all the counts should be in the table
    .start()
)


new_rating_df.writeStream\
  .format("delta")\
  .outputMode("append")\
  .option("checkpointLocation", "/tmp/delta/ratings/_checkpoints/")\
  .start("/delta/ratings")



Out[71]: <pyspark.sql.streaming.query.StreamingQuery at 0x7f7678908bb0>

In [0]:
# %fs
# rm -r /delta/ratings
# ls /delta/ratings

at com.databricks.backend.daemon.data.client.DbfsClient.send0(DbfsClient.scala:121)
	at com.databricks.backend.daemon.data.client.DbfsClient.sendIdempotent(DbfsClient.scala:69)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV1.listStatus(DatabricksFileSystemV1.scala:179)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystem.listStatus(DatabricksFileSystem.scala:161)
	at com.databricks.backend.daemon.dbutils.FSUtils.lsWithLimit(DBUtilsCore.scala:254)
	at com.databricks.backend.daemon.dbutils.FSUtils.$anonfun$lsImpl$2(DBUtilsCore.scala:223)
	at com.databricks.backend.daemon.dbutils.FSUtils.withFsSafetyCheck(DBUtilsCore.scala:145)
	at com.databricks.backend.daemon.dbutils.FSUtils.$anonfun$lsImpl$1(DBUtilsCore.scala:221)
	at com.databricks.backend.daemon.dbutils.FSUtils.checkPermission(DBUtilsCore.scala:140)
	at com.databricks.backend.daemon.dbutils.FSUtils.lsImpl(DBUtilsCore.scala:221)
	at com.databricks.backend.daemon.dbutils.FSUtils.$anonfun$ls$1(DBUtilsCore.scala:211)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$1(UsageLogging.scala:560)
	at com.databricks.logging.UsageLogging.executeThunkAndCaptureResultTags$1(UsageLogging.scala:656)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperationWithResultTags$4(UsageLogging.scala:677)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:414)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.AttributionContext$.withValue(AttributionContext.scala:158)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:412)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:409)
	at com.databricks.backend.daemon.dbutils.FSUtils.withAttributionContext(DBUtilsCore.scala:69)
	at com.databricks.logging.UsageLogging.withAttributionTags(UsageLogging.scala:457)
	at com.databricks.logging.UsageLogging.withAttributionTags$(UsageLogging.scala:442)
	at com.databricks.backend.daemon.dbutils.FSUtils.withAttributionTags(DBUtilsCore.scala:69)
	at com.databricks.logging.UsageLogging.recordOperationWithResultTags(UsageLogging.scala:651)
	at com.databricks.logging.UsageLogging.recordOperationWithResultTags$(UsageLogging.scala:569)
	at com.databricks.backend.daemon.dbutils.FSUtils.recordOperationWithResultTags(DBUtilsCore.scala:69)
	at com.databricks.logging.UsageLogging.recordOperation(UsageLogging.scala:560)
	at com.databricks.logging.UsageLogging.recordOperation$(UsageLogging.scala:528)
	at com.databricks.backend.daemon.dbutils.FSUtils.recordOperation(DBUtilsCore.scala:69)
	at com.databricks.backend.daemon.dbutils.FSUtils.recordDbutilsFsOp(DBUtilsCore.scala:133)
	at com.databricks.backend.daemon.dbutils.FSUtils.ls(DBUtilsCore.scala:211)
	at com.databricks.dbutils_v1.impl.DbfsUtilsImpl.ls(DbfsUtilsImpl.scala:67)
	at $line6510185b46fa45cca3576b9b3afd3a9b29.$read$$iw$$iw$$iw$$iw$$iw$$iw.<init>(command-2042515557813446:1)
	at $line6510185b46fa45cca3576b9b3afd3a9b29.$read$$iw$$iw$$iw$$iw$$iw.<init>(command-2042515557813446:43)
	at $line6510185b46fa45cca3576b9b3afd3a9b29.$read$$iw$$iw$$iw$$iw.<init>(command-2042515557813446:45)
	at $line6510185b46fa45cca3576b9b3afd3a9b29.$read$$iw$$iw$$iw.<init>(command-2042515557813446:47)
	at $line6510185b46fa45cca3576b9b3afd3a9b29.$read$$iw$$iw.<init>(command-2042515557813446:49)
	at $line6510185b46fa45cca3576b9b3afd3a9b29.$read$$iw.<init>(command-2042515557813446:51)
	at $line6510185b46fa45cca3576b9b3afd3a9b29.$read.<init>(command-2042515557813446:53)
	at $line6510185b46fa45cca3576b9b3afd3a9b29.$read$.<init>(command-2042515557813446:57)
	at $line6510185b46fa45cca3576b9b3afd3a9b29.$read$.<clinit>(command-2042515557813446)
	at $line6510185b46fa45cca3576b9b3afd3a9b29.$eval$.$print$lzycompute(<notebook>:7)
	at $line6510185b46fa45cca3576b9b3afd3a9b29.$eval$.$print(<notebook>:6)
	at $line6510185b46fa45cca3576b9b3afd3a9b29.$eval.$print(<notebook>)
	at sun.reflect.NativeMethodAccessorImpl.i